# 🚕 The Taxi Problem (Taxi-v3) 🚕

### (Reinforcement Learning Solution)

## Description:
There are four designated locations in the grid world indicated by R(ed), G(reen), Y(ellow), and B(lue). When the episode starts, the taxi starts off at a random square and the passenger is at a random location. The taxi drives to the passenger's location, picks up the passenger, drives to the passenger's destination (another one of the four specified locations), and then drops off the passenger. Once the passenger is dropped off, the episode ends.

[Openai Link](https://gym.openai.com/envs/Taxi-v3/)  
[Github Link](https://github.com/openai/gym/blob/master/gym/envs/toy_text/taxi.py)

## Observations:
There are 500 discrete states since there are 25 taxi positions, 5 possible locations of the passenger (including the case when the passenger is in the taxi), and 4 destination locations. 

## Passenger locations:
- 0: R(ed)
- 1: G(reen)
- 2: Y(ellow)
- 3: B(lue)
- 4: in taxi

## Destinations:
- 0: R(ed)
- 1: G(reen)
- 2: Y(ellow)
- 3: B(lue)

## Actions:
There are 6 discrete deterministic actions:
- 0: move south
- 1: move north
- 2: move east
- 3: move west
- 4: pickup passenger
- 5: drop off passenger

## Import Packages

In [2]:
import gym
import random
import numpy as np
import pandas as pd
from pylab import plt
from IPython import display
plt.style.use('seaborn')
import warnings; warnings.simplefilter('ignore')
from collections import deque

## Setting seeds

In [3]:
def set_seeds(seed=42):
    random.seed(seed)
    np.random.seed(seed)
    env.seed(seed)

## Environment

In [4]:
env = gym.make('Taxi-v3')

## Action Space

In [5]:
env.action_space  # type of action space

Discrete(6)

In [6]:
env.action_space.n  # number of discrete actions

6

In [7]:
env.action_space.sample()  # sample action

4

In [8]:
[env.action_space.sample() for _ in range(10)]

[3, 3, 1, 0, 2, 0, 3, 4, 0, 1]

## Observation Space

In [9]:
np.set_printoptions(precision=4, suppress=True)

In [10]:
env.observation_space  # type of observation space

Discrete(500)

In [11]:
o = env.reset()
o

212

In [12]:
# (taxi row, taxi column, passenger location, destination location)
list(env.decode(o))

[2, 0, 3, 0]

In [13]:
env.decode(o)

## Taking Action

The **blue** letter represents the current passenger pick-up location, and the **purple** letter is the current destination.

In [14]:
env.render()

+---------+
|R: | : :G|
| : | : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+



To save the render into a variable you use the 'ansi' option `env.render(mode='ansi')`

In [15]:
r = env.render(mode='ansi')
print(r)

+---------+
|R: | : :G|
| : | : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+




In [16]:
a = env.action_space.sample()  # random action
a

2

In [17]:
r = env.step(a)  # taking action, capturing new observations
r  # (observation, reward, done, info)

(232, -1, False, {'prob': 1.0})

In [18]:
env.render()

+---------+
|R: | : :G|
| : | : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+
  (East)


In [19]:
env.step(1)

(132, -1, False, {'prob': 1.0})

In [20]:
env.render()

+---------+
|R: | : :G|
| : | : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+
  (North)


In [21]:
env.step(1)

(32, -1, False, {'prob': 1.0})

In [22]:
env.render()

+---------+
|R: | : :G|
| : | : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+
  (North)


## Random Algorithm

In [23]:
##
## Code copied from 
## https://www.learndatasci.com/tutorials/reinforcement-q-learning-scratch-python-openai-gym/
##

#env.s = 328  # set environment to illustration's state

epochs = 0
penalties, reward = 0, 0

frames = [] # for animation

done = False

while not done:
    action = env.action_space.sample()
    state, reward, done, info = env.step(action)

    if reward == -10:
        penalties += 1
    
    # Put each rendered frame into dict for animation
    frames.append({
        'frame': env.render(mode='ansi'),
        'state': state,
        'action': action,
        'reward': reward
        }
    )

    epochs += 1
    
    
print("Timesteps taken: {}".format(epochs))
print("Penalties incurred: {}".format(penalties))

Timesteps taken: 197
Penalties incurred: 68


In [25]:
from IPython.display import clear_output
from time import sleep

def print_frames(frames):
    for i, frame in enumerate(frames):
        clear_output(wait=True)
        print(frame['frame'])
        print(f"Timestep: {i + 1}")
        print(f"State: {frame['state']}")
        print(f"Action: {frame['action']}")
        print(f"Reward: {frame['reward']}")
        sleep(.5)
        
#print_frames(frames)

In [26]:
env.action_space.n

6

## Q Learning

In [36]:
class DQLAgent:
    def __init__(self):
        self.epsilon = 1.0  # initial epsilon
        self.epsilon_min = 0.01  # minimal epsilon
        self.epsilon_decay = 0.995  # epsilon decay
        self.gamma = 0.95  # discount factor
        self.batch_size = 128  # batch size for replay
        self.max_treward = -1e6
        self.averages = list()
        self.memory = deque(maxlen=2000)  # fixed memory
        self.osn = 1
        self.qtable = np.ones((env.observation_space.n, env.action_space.n))/env.action_space.n
        
        
    def act(self, state):
        if random.random() <= self.epsilon:
            return env.action_space.sample()
        action = np.argmax(self.qtable[state,:])
        return action  # choose action with highest value
    
    def replay(self):
        batch = random.sample(self.memory, self.batch_size)

        for state, action, reward, next_state, done in batch:
            if not done:
                self.qtable[state, action] +=  reward + self.gamma * \
                np.max(self.qtable[next_state, :]) - self.qtable[state, action]
            
        if self.epsilon > self.epsilon_min:
            self.epsilon *= self.epsilon_decay

    
    
    def learn(self, episodes):
        trewards = []
        i = 0
        for e in range(1, episodes + 1):
            state = env.reset()
            treward = 0
            for _ in range(5000):
                action = self.act(state)
                next_state, reward, done, info = env.step(action)
                self.memory.append([state, action, reward,
                                     next_state, done])
                state = next_state
                treward += float(reward)
                if done:
                    trewards.append(treward)
                    av = sum(trewards[-25:]) / 25
                    self.averages.append(av)
                    self.max_treward = max(self.max_treward, treward)
                    templ = 'episode: {:4d}/{} | treward: {:7.1f} | '
                    templ += 'av: {:7.1f} | max: {:7.1f}'
                    print(templ.format(e, episodes, treward, av,
                                       self.max_treward), end='\r')
                    break
            if len(self.memory) > self.batch_size:
                self.replay()
            if treward > 200:
                break
        print()
                
    def test(self, episodes):
        trewards = []
        for e in range(1, episodes + 1):
            state = env.reset()
            treward = 0
            for _ in range(1001):
                action = np.argmax(self.qtable[state,:])
                next_state, reward, done, info = env.step(action)
                state = next_state
                treward += float(reward)
                if done:
                    trewards.append(treward)
                    print('episode: {:4d}/{} | treward: {:7.1f}'
                          .format(e, episodes, treward), end='\r')
                    break
        return trewards

In [37]:
set_seeds(100)
agent = DQLAgent()

In [38]:
episodes = 5000

In [39]:
%time agent.learn(episodes)

episode: 5000/5000 | treward:     7.0 | av:     7.0 | max:    15.0
CPU times: user 1min 11s, sys: 14.8 s, total: 1min 26s
Wall time: 1min 11s


In [40]:
agent.qtable

array([[  0.1667,   0.1667,   0.1667,   0.1667,   0.1667,   0.1667],
       [ -7.9255,  -7.9255,  -7.9255,  -7.9255,  -7.29  , -16.9255],
       [ -5.1756,  -5.1756,  -5.1756,  -5.1756,  -4.3954, -14.1756],
       ...,
       [ -4.3954,  -3.5741,  -4.3954,  -3.5741, -11.7096, -13.3954],
       [ -5.9169,  -5.9169,  -5.9169,  -6.621 , -14.1756, -14.9169],
       [ -1.7996,  -1.7996,  -1.7996,  -0.8417, -10.7996, -10.7996]])

In [41]:
trewards = agent.test(20)

In [42]:
sum(trewards) / len(trewards)

7.45

In [43]:
epochs = 0
treward, reward = 0, 0
frames = [] # for animation
state = env.reset()
done = False

while not done:
    action = np.argmax(agent.qtable[state,:])
    state, reward, done, info = env.step(action)
    
    treward += float(reward)

    # Put each rendered frame into dict for animation
    frames.append({
        'frame': env.render(mode='ansi'),
        'state': state,
        'action': action,
        'reward': reward
        }
    )

    epochs += 1
    
    
print("Timesteps taken: {}".format(epochs))
print("Total Reward: {}".format(treward))

Timesteps taken: 13
Total Reward: 8.0


In [44]:
print_frames(frames)

+---------+
|R: | : :G|
| : | : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+
  (Dropoff)

Timestep: 13
State: 475
Action: 5
Reward: 20
